In [6]:
import tensorflow as tf

from tensorflow.keras.datasets import mnist

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


In [7]:
# Step 1: Load and preprocess the data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Splitting the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=5000, random_state=42)

In [8]:
print("Shape of x_train:", x_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of x_test:", x_test.shape)
print("Shape of y_test:", y_test.shape)
print("Shape of x_val:", x_val.shape)
print("Shape of y_val:", y_val.shape)

Shape of x_train: (55000, 28, 28)
Shape of y_train: (55000,)
Shape of x_test: (10000, 28, 28)
Shape of y_test: (10000,)
Shape of x_val: (5000, 28, 28)
Shape of y_val: (5000,)


In [9]:
# Step 2: Building the Model
model = Sequential()
model.add(Input(shape=(28, 28)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer=GlorotUniform()))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu', kernel_initializer=GlorotUniform()))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

# Check if the model is a classification model
is_classification_model = model.layers[-1].activation == tf.keras.activations.softmax

# Step 3: Setting up early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Step 4: Integrating TensorBoard
tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1)

# Step 5: Baseline with Logistic Regression
logreg = LogisticRegression(max_iter=1000)  # Increased max_iter to avoid convergence warning
logreg.fit(x_train.reshape(-1, 28*28), y_train)
y_pred = logreg.predict(x_test.reshape(-1, 28*28))
baseline_accuracy = accuracy_score(y_test, y_pred)

# Training the Model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[early_stopping, tensorboard_callback], epochs=100, batch_size=32)

# Evaluating the Model
test_loss, test_accuracy = model.evaluate(x_test, y_test)

# Printing the Results
print("Baseline Accuracy:", baseline_accuracy)
print("Test Accuracy:", test_accuracy)

Epoch 1/100


2024-05-22 14:16:46.717250: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 172480000 exceeds 10% of free system memory.


1719/1719 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8330 - loss: 0.5583 - val_accuracy: 0.9562 - val_loss: 0.1395
Epoch 2/100
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9375 - loss: 0.2005 - val_accuracy: 0.9662 - val_loss: 0.1069
Epoch 3/100
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9499 - loss: 0.1649 - val_accuracy: 0.9706 - val_loss: 0.1013
Epoch 4/100
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9585 - loss: 0.1382 - val_accuracy: 0.9718 - val_loss: 0.0912
Epoch 5/100
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9570 - loss: 0.1371 - val_accuracy: 0.9742 - val_loss: 0.0830
Epoch 6/100
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9645 - loss: 0.1188 - val_accuracy: 0.9762 - val_loss: 0.0776
Epoch 7/100
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9654 - loss: 0.1110 - val_accuracy: 0.9780 - val_loss: 0.0755
Epoch 8/100
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9697 - loss: 0.0990 - val_

Voici une traduction en français du texte en markdown :

```markdown
Étape 1 : Chargement et prétraitement des données
- La fonction `mnist.load_data()` est utilisée pour charger l'ensemble de données MNIST, qui contient des chiffres écrits à la main.
- Les données sont divisées en ensembles d'entraînement et de test, avec `x_train` et `y_train` représentant les images d'entrée et leurs étiquettes correspondantes pour l'entraînement, et `x_test` et `y_test` représentant les images d'entrée et leurs étiquettes correspondantes pour les tests.
- Les valeurs des pixels des images sont normalisées en les divisant par 255.0, ce qui met les valeurs à l'échelle entre 0 et 1.

Étape 2 : Construction du modèle
- Le modèle `Sequential` est créé en utilisant `model = Sequential()`.
- La forme d'entrée du modèle est définie comme `(28, 28)` en utilisant `Flatten(input_shape=(28, 28))`, ce qui aplatit les images d'entrée 2D en un tableau 1D.
- Deux couches entièrement connectées avec 128 et 64 unités, respectivement, sont ajoutées en utilisant des couches `Dense` avec la fonction d'activation `relu`.
- Des couches `BatchNormalization` sont ajoutées après chaque couche `Dense` pour normaliser les activations.
- Des couches `Dropout` avec un taux de désactivation de 0.2 sont ajoutées après chaque couche `BatchNormalization` pour éviter le surajustement.
- La couche de sortie finale avec 10 unités et la fonction d'activation `softmax` est ajoutée en utilisant `Dense(10, activation='softmax')`, car il s'agit d'un problème de classification multi-classes avec 10 classes.

Étape 3 : Configuration de l'arrêt précoce
- Un rappel `EarlyStopping` est créé avec le paramètre `monitor` défini sur `'val_accuracy'`, ce qui signifie qu'il surveillera la précision de validation.
- Le paramètre `patience` est défini sur 3, ce qui signifie que l'entraînement s'arrêtera si la précision de validation n'améliore pas pendant 3 époques consécutives.
- Le paramètre `restore_best_weights` est défini sur `True`, ce qui signifie que les poids du modèle seront restaurés aux meilleurs poids trouvés pendant l'entraînement.

Étape 4 : Intégration de TensorBoard
- Un rappel `TensorBoard` est créé avec le paramètre `log_dir` défini sur `'./logs'`, ce qui spécifie le répertoire où les journaux seront enregistrés.
- Le paramètre `histogram_freq` est défini sur 1, ce qui signifie que les histogrammes des poids et des biais du modèle seront calculés et enregistrés à chaque époque.

Étape 5 : Baseline avec la régression logistique
- Un modèle `LogisticRegression` est créé et entraîné en utilisant les données d'entraînement.
- Les tableaux `x_train` et `x_test` sont remodelés en tableaux 2D en utilisant `reshape(-1, 28*28)` pour correspondre à la forme d'entrée attendue par `LogisticRegression`.
- La méthode `predict` est utilisée pour effectuer des prédictions sur les données de test, et la précision des prédictions est calculée à l'aide de `accuracy_score` et stockée dans la variable `baseline_accuracy`.

Entraînement du modèle
- Le modèle est compilé avec l'optimiseur `adam` et la fonction de perte `sparse_categorical_crossentropy`, qui convient aux problèmes de classification multi-classes.
- La méthode `fit` est appelée pour entraîner le modèle en utilisant les données d'entraînement.
- Les données de validation `(x_val, y_val)` sont utilisées pour surveiller la précision de validation pendant l'entraînement.
- Les rappels `early_stopping` et `tensorboard_callback` sont transmis au paramètre `callbacks` pour activer l'arrêt précoce et l'intégration de TensorBoard.
- L'entraînement est effectué pendant 10 époques.

Évaluation du modèle
- La méthode `evaluate` est appelée pour évaluer le modèle entraîné sur les données de test.
- La perte de test et la précision de test sont stockées dans les variables `test_loss` et `test_accuracy`, respectivement.

Affichage des résultats
- La précision de référence et la précision de test sont affichées à l'aide de la fonction `print`.

```
